# 1 MLM

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/My Drive/my_code/bert-and-text-classification

/content/drive/My Drive/my_code/bert-and-text-classification


In [4]:
# !pip install torch torchvision

In [5]:
# !pip install pytorch_transformers

预测mask

In [6]:
import torch
from pytorch_transformers import BertTokenizer

In [7]:
model_name = 'bert-base-chinese'  # 指定实验需下载的预训练模型参数

# BERT 在预训练中引入了 [CLS] 和 [SEP] 标记句子的开头和结尾
samples = ['[CLS] 中国的首都是哪里？ [SEP] 北京是 [MASK] 国的首都。 [SEP]']  # 准备输入模型的语句

In [8]:
tokenizer = BertTokenizer.from_pretrained(model_name)

In [9]:
tokenized_text = [tokenizer.tokenize(i) for i in samples]
tokenized_text

[['[CLS]',
  '中',
  '国',
  '的',
  '首',
  '都',
  '是',
  '哪',
  '里',
  '？',
  '[SEP]',
  '北',
  '京',
  '是',
  '[MASK]',
  '国',
  '的',
  '首',
  '都',
  '。',
  '[SEP]']]

In [10]:
input_ids = [tokenizer.convert_tokens_to_ids(i) for i in tokenized_text]
input_ids

[[101,
  704,
  1744,
  4638,
  7674,
  6963,
  3221,
  1525,
  7027,
  8043,
  102,
  1266,
  776,
  3221,
  103,
  1744,
  4638,
  7674,
  6963,
  511,
  102]]

In [11]:
input_ids = torch.LongTensor(input_ids)
input_ids

tensor([[ 101,  704, 1744, 4638, 7674, 6963, 3221, 1525, 7027, 8043,  102, 1266,
          776, 3221,  103, 1744, 4638, 7674, 6963,  511,  102]])

In [12]:
from pytorch_transformers import BertForMaskedLM

# 读取预训练模型
model = BertForMaskedLM.from_pretrained(model_name, cache_dir="./")

In [13]:
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [14]:
outputs = model(input_ids)
outputs

(tensor([[[ -7.9952,  -7.9039,  -7.9825,  ...,  -6.9982,  -7.0039,  -7.0533],
          [ -7.3028,  -7.6208,  -7.4104,  ...,  -5.9638,  -6.7491,  -3.6037],
          [-18.8863, -18.8394, -18.4806,  ..., -14.1091,  -9.2716, -10.0511],
          ...,
          [-15.1031, -15.8225, -14.4563,  ..., -10.7002, -10.1100,  -6.4397],
          [-11.3589, -11.2920, -10.8067,  ...,  -8.5306,  -5.9084,  -5.8770],
          [ -9.2334,  -9.1561,  -9.3433,  ...,  -6.9585,  -6.7155,  -6.3854]]],
        grad_fn=<AddBackward0>),)

In [15]:
prediction_scores = outputs[0]

In [16]:
prediction_scores.shape

torch.Size([1, 21, 21128])

In [17]:
prediction_scores

tensor([[[ -7.9952,  -7.9039,  -7.9825,  ...,  -6.9982,  -7.0039,  -7.0533],
         [ -7.3028,  -7.6208,  -7.4104,  ...,  -5.9638,  -6.7491,  -3.6037],
         [-18.8863, -18.8394, -18.4806,  ..., -14.1091,  -9.2716, -10.0511],
         ...,
         [-15.1031, -15.8225, -14.4563,  ..., -10.7002, -10.1100,  -6.4397],
         [-11.3589, -11.2920, -10.8067,  ...,  -8.5306,  -5.9084,  -5.8770],
         [ -9.2334,  -9.1561,  -9.3433,  ...,  -6.9585,  -6.7155,  -6.3854]]],
       grad_fn=<AddBackward0>)

In [18]:
import numpy as np
prediction_scores[0].shape

torch.Size([21, 21128])

In [19]:
sample = prediction_scores[0].detach().numpy()
sample.shape

(21, 21128)

In [20]:
pred = np.argmax(sample, axis=1)
pred.shape

(21,)

In [21]:
tokenizer.convert_ids_to_tokens(pred)[14]

'中'

# 2 Next Sentence

预测下一个句子

In [22]:
samples = ["[CLS]今天天气怎么样？[SEP]今天天气很好。[SEP]", "[CLS]小明今年几岁了？[SEP]小明爱吃西瓜。[SEP]"]
tokenizer = BertTokenizer.from_pretrained(model_name)

In [23]:
tokenized_text = [tokenizer.tokenize(i) for i in samples]
tokenized_text

[['[CLS]',
  '今',
  '天',
  '天',
  '气',
  '怎',
  '么',
  '样',
  '？',
  '[SEP]',
  '今',
  '天',
  '天',
  '气',
  '很',
  '好',
  '。',
  '[SEP]'],
 ['[CLS]',
  '小',
  '明',
  '今',
  '年',
  '几',
  '岁',
  '了',
  '？',
  '[SEP]',
  '小',
  '明',
  '爱',
  '吃',
  '西',
  '瓜',
  '。',
  '[SEP]']]

In [24]:
input_ids = [tokenizer.convert_tokens_to_ids(i) for i in tokenized_text]
input_ids

[[101,
  791,
  1921,
  1921,
  3698,
  2582,
  720,
  3416,
  8043,
  102,
  791,
  1921,
  1921,
  3698,
  2523,
  1962,
  511,
  102],
 [101,
  2207,
  3209,
  791,
  2399,
  1126,
  2259,
  749,
  8043,
  102,
  2207,
  3209,
  4263,
  1391,
  6205,
  4478,
  511,
  102]]

In [25]:
input_ids = torch.LongTensor(input_ids)
input_ids

tensor([[ 101,  791, 1921, 1921, 3698, 2582,  720, 3416, 8043,  102,  791, 1921,
         1921, 3698, 2523, 1962,  511,  102],
        [ 101, 2207, 3209,  791, 2399, 1126, 2259,  749, 8043,  102, 2207, 3209,
         4263, 1391, 6205, 4478,  511,  102]])

In [26]:
segments_ids = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]]

In [27]:
segments_tensors = torch.tensor(segments_ids)
segments_tensors

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]])

In [28]:
from pytorch_transformers import BertForNextSentencePrediction

model = BertForNextSentencePrediction.from_pretrained(
    model_name, cache_dir="./")
model.eval()

BertForNextSentencePrediction(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [29]:
outputs = model(input_ids)
seq_relationship_scores = outputs[0]
seq_relationship_scores

tensor([[3.0364, 1.0698],
        [2.8612, 0.9533]], grad_fn=<AddmmBackward>)

In [30]:
sample = seq_relationship_scores.detach().numpy()
pred = np.argmax(sample, axis=1)
pred

array([0, 0])

# 3 text classification

从预训练好的 BERT 模型中提取特征向量，即 Feature Extraction 方法。 \
将下游任务模型添加到 BERT 模型之后，然后使用下游任务的训练集对进行训练，即 Fine-Tuning 方法。

In [31]:
with open('clothing_comment/negdata.txt', 'r', encoding='utf-8') as f:
    neg_data = f.read()
with open('clothing_comment/posdata.txt', 'r', encoding='utf-8') as f:
    pos_data = f.read()

In [32]:
neg_datalist = neg_data.split('\n')
pos_datalist = pos_data.split('\n')
len(neg_datalist), len(pos_datalist)

(1500, 1500)

In [33]:
import numpy as np

dataset = np.array(pos_datalist + neg_datalist)
labels = np.array([1] * len(pos_datalist) + [0] * len(neg_datalist))
len(dataset)  # 共 3000 条数据

3000

In [34]:
np.random.seed(10)
mix_index = np.random.choice(3000, 3000)
dataset = dataset[mix_index]
labels = labels[mix_index]
len(dataset), len(labels)

(3000, 3000)

In [35]:
TRAINSET_SIZE = 2500
EVALSET_SIZE = 500

train_samples = dataset[:TRAINSET_SIZE]  # 2500 条数据
train_labels = labels[:TRAINSET_SIZE]
eval_samples = dataset[TRAINSET_SIZE:TRAINSET_SIZE+EVALSET_SIZE]  # 500 条数据
eval_labels = labels[TRAINSET_SIZE:TRAINSET_SIZE+EVALSET_SIZE]

len(train_samples), len(eval_samples)

(2500, 500)

In [36]:
def get_dummies(l, size=2):
    res = list()
    for i in l:
        tmp = [0] * size
        tmp[i] = 1
        res.append(tmp)
    return res

In [37]:
from torch.utils.data import DataLoader, TensorDataset

tokenized_text = [tokenizer.tokenize(i) for i in train_samples]
tokenized_text

[['衣', '服', '有', '点', '味', '道', '，', '其', '他', '服', '务', '还', '好', '啦'],
 ['质',
  '量',
  '不',
  '错',
  '，',
  '穿',
  '起',
  '来',
  '很',
  '舒',
  '服',
  '，',
  '尺',
  '码',
  '也',
  '合',
  '适'],
 ['物',
  '流',
  '很',
  '给',
  '力',
  '，',
  '穿',
  '着',
  '很',
  '舒',
  '服',
  '，',
  '也',
  '很',
  '好',
  '看',
  '。',
  '布',
  '料',
  '也',
  '相',
  '当',
  '不',
  '错',
  '，',
  '特',
  '别',
  '是',
  '客',
  '服',
  ',',
  '热',
  '情',
  '好',
  '客',
  '，',
  '服',
  '务',
  '态',
  '度',
  '一',
  '流'],
 ['包',
  '装',
  '精',
  '美',
  '，',
  '很',
  '上',
  '档',
  '次',
  '。',
  '做',
  '工',
  '精',
  '致',
  '，',
  '质',
  '量',
  '一',
  '流',
  '。',
  '颜',
  '色',
  '鲜',
  '艳',
  '零',
  '色',
  '差',
  '，',
  '尺',
  '寸',
  '大',
  '小',
  '刚',
  '刚',
  '好',
  '，',
  '版',
  '型',
  '很',
  '正',
  '，',
  '很',
  '显',
  '腰',
  '身',
  '，',
  '布',
  '料',
  '柔',
  '软',
  '舒',
  '适',
  '，',
  '很',
  '是',
  '喜',
  '欢',
  '。',
  '大',
  '爱',
  '!',
  '!',
  '!',
  '下',
  '次',
  '还',
  '会',
  '在',
  '来',
  '的'],
 ['一', '颗', '星', '都',

In [38]:
input_ids = [tokenizer.convert_tokens_to_ids(i) for i in tokenized_text]
input_ids

[[6132,
  3302,
  3300,
  4157,
  1456,
  6887,
  8024,
  1071,
  800,
  3302,
  1218,
  6820,
  1962,
  1568],
 [6574,
  7030,
  679,
  7231,
  8024,
  4959,
  6629,
  3341,
  2523,
  5653,
  3302,
  8024,
  2223,
  4772,
  738,
  1394,
  6844],
 [4289,
  3837,
  2523,
  5314,
  1213,
  8024,
  4959,
  4708,
  2523,
  5653,
  3302,
  8024,
  738,
  2523,
  1962,
  4692,
  511,
  2357,
  3160,
  738,
  4685,
  2496,
  679,
  7231,
  8024,
  4294,
  1166,
  3221,
  2145,
  3302,
  117,
  4178,
  2658,
  1962,
  2145,
  8024,
  3302,
  1218,
  2578,
  2428,
  671,
  3837],
 [1259,
  6163,
  5125,
  5401,
  8024,
  2523,
  677,
  3440,
  3613,
  511,
  976,
  2339,
  5125,
  5636,
  8024,
  6574,
  7030,
  671,
  3837,
  511,
  7582,
  5682,
  7831,
  5683,
  7439,
  5682,
  2345,
  8024,
  2223,
  2189,
  1920,
  2207,
  1157,
  1157,
  1962,
  8024,
  4276,
  1798,
  2523,
  3633,
  8024,
  2523,
  3227,
  5587,
  6716,
  8024,
  2357,
  3160,
  3382,
  6763,
  5653,
  6844,
  8024,
  2

In [39]:
input_labels = get_dummies(train_labels)  # 使用 get_dummies 函数转换标签
input_labels

[[0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 

In [40]:
for j in range(len(input_ids)):
    # 将样本数据填充至长度为 512
    i = input_ids[j]
    if len(i) != 512:
        input_ids[j].extend([0]*(512 - len(i)))

In [41]:
# 构建数据集和数据迭代器，设定 batch_size 大小为 4
train_set = TensorDataset(torch.LongTensor(input_ids),
                          torch.FloatTensor(input_labels))

In [42]:
train_loader = DataLoader(dataset=train_set,
                          batch_size=4, #4
                          shuffle=True)

In [43]:
train_loader

In [44]:
tokenized_text = [tokenizer.tokenize(i) for i in eval_samples]
input_ids = [tokenizer.convert_tokens_to_ids(i) for i in tokenized_text]
input_labels = eval_labels

for j in range(len(input_ids)):
    i = input_ids[j]
    if len(i) != 512:
        input_ids[j].extend([0]*(512 - len(i)))

eval_set = TensorDataset(torch.LongTensor(input_ids),
                         torch.FloatTensor(input_labels))
eval_loader = DataLoader(dataset=eval_set,
                         batch_size=1,
                         shuffle=True)
eval_loader

In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [46]:
import torch.nn as nn
import torch.nn.functional as F
from pytorch_transformers import BertModel


class fn_cls(nn.Module):
    def __init__(self):
        super(fn_cls, self).__init__()
        self.model = BertModel.from_pretrained(model_name, cache_dir="./")
        self.model.to(device)
        self.dropout = nn.Dropout(0.1)
        self.l1 = nn.Linear(768, 2)

    def forward(self, x, attention_mask=None):
        outputs = self.model(x, attention_mask=attention_mask)
        x = outputs[1]  # 取池化后的结果 batch * 768
        x = x.view(-1, 768)
        x = self.dropout(x)
        x = self.l1(x)
        return x

In [47]:
from torch import optim

cls = fn_cls()
cls.to(device)
cls.train()

fn_cls(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [48]:
criterion = nn.BCELoss()
sigmoid = nn.Sigmoid()
optimizer = optim.Adam(cls.parameters(), lr=1e-5)

In [49]:
def predict(logits):
    res = torch.argmax(logits, 1)
    return res

In [50]:
from torch.autograd import Variable
import time

pre = time.time()

accumulation_steps = 8
epoch = 3

for i in range(epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).to(device), Variable(
            target.view(-1, 2)).to(device)

        mask = []
        for sample in data:
            mask.append([1 if i != 0 else 0 for i in sample])
        mask = torch.Tensor(mask).to(device)

        output = cls(data, attention_mask=mask)
        pred = predict(output)

        loss = criterion(sigmoid(output).view(-1, 2), target)

        # 梯度积累
        loss = loss/accumulation_steps
        loss.backward()

        if((batch_idx+1) % accumulation_steps) == 0:
            # 每 8 次更新一下网络中的参数
            optimizer.step()
            optimizer.zero_grad()

        if ((batch_idx+1) % accumulation_steps) == 1:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss:{:.6f}'.format(
                i+1, batch_idx, len(train_loader), 100. *
                batch_idx/len(train_loader), loss.item()
            ))
        if batch_idx == len(train_loader)-1:
            # 在每个 Epoch 的最后输出一下结果
            print('labels:', target)
            print('pred:', pred)

print('训练时间：', time.time()-pre)

Train Epoch: 1 [0/625 (0%)]	Loss:0.091476
Train Epoch: 1 [8/625 (1%)]	Loss:0.084410
Train Epoch: 1 [16/625 (3%)]	Loss:0.082216
Train Epoch: 1 [24/625 (4%)]	Loss:0.090585
Train Epoch: 1 [32/625 (5%)]	Loss:0.067588
Train Epoch: 1 [40/625 (6%)]	Loss:0.085150
Train Epoch: 1 [48/625 (8%)]	Loss:0.099709
Train Epoch: 1 [56/625 (9%)]	Loss:0.084623
Train Epoch: 1 [64/625 (10%)]	Loss:0.064873
Train Epoch: 1 [72/625 (12%)]	Loss:0.072833
Train Epoch: 1 [80/625 (13%)]	Loss:0.101045
Train Epoch: 1 [88/625 (14%)]	Loss:0.070708
Train Epoch: 1 [96/625 (15%)]	Loss:0.085251
Train Epoch: 1 [104/625 (17%)]	Loss:0.084042
Train Epoch: 1 [112/625 (18%)]	Loss:0.072067
Train Epoch: 1 [120/625 (19%)]	Loss:0.086615
Train Epoch: 1 [128/625 (20%)]	Loss:0.074689
Train Epoch: 1 [136/625 (22%)]	Loss:0.058467
Train Epoch: 1 [144/625 (23%)]	Loss:0.064813
Train Epoch: 1 [152/625 (24%)]	Loss:0.055440
Train Epoch: 1 [160/625 (26%)]	Loss:0.055010
Train Epoch: 1 [168/625 (27%)]	Loss:0.062606
Train Epoch: 1 [176/625 (28%)]	Lo

In [51]:
from tqdm.notebook import tqdm

cls.eval()

correct = 0
total = 0

for batch_idx, (data, target) in enumerate(tqdm(eval_loader)):
    data = data.to(device)
    target = target.long().to(device)

    mask = []
    for sample in data:
        mask.append([1 if i != 0 else 0 for i in sample])
    mask = torch.Tensor(mask).to(device)

    output = cls(data, attention_mask=mask)
    pred = predict(output)

    correct += (pred == target).sum().item()
    total += len(data)

# 准确率应该达到百分之 90 以上
print('正确分类的样本数：{}，样本总数：{}，准确率：{:.2f}%'.format(
    correct, total, 100.*correct/total))


正确分类的样本数：472，样本总数：500，准确率：94.40%


In [52]:
test_samples = ['东西很好，好评！', '东西不好，差评！']

In [53]:
cls.eval()
tokenized_text = [tokenizer.tokenize(i) for i in test_samples]
input_ids = [tokenizer.convert_tokens_to_ids(i) for i in tokenized_text]
input_ids = torch.LongTensor(input_ids).cuda()

mask = torch.ones_like(input_ids).to(device)

output = cls(input_ids, attention_mask=mask)
pred = predict(output)
pred

tensor([1, 0], device='cuda:0')